# Chatwoot Regions Setup

This notebook sets up a custom attribute for regions in Chatwoot. It will:
1. Load environment variables
2. Initialize Chatwoot handler
3. Create a custom attribute definition for regions and create teams if they don't exist

In [ ]:
import logging
import os

import httpx
from dotenv import load_dotenv

from app.api.chatwoot import ChatwootHandler

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Get Chatwoot configuration from environment
CHATWOOT_API_URL = os.getenv("CHATWOOT_API_URL_DEV")
CHATWOOT_API_KEY = os.getenv("CHATWOOT_API_KEY_DEV")
CHATWOOT_ACCOUNT_ID = os.getenv("CHATWOOT_ACCOUNT_ID")

# Verify environment variables
if not all([CHATWOOT_API_URL, CHATWOOT_API_KEY, CHATWOOT_ACCOUNT_ID]):
    raise ValueError("Missing required Chatwoot environment variables. Please check your .env file.")

logger.info(f"Using Chatwoot API URL: {CHATWOOT_API_URL}")
logger.info(f"Using Chatwoot Account ID: {CHATWOOT_ACCOUNT_ID}")

In [ ]:
# Initialize Chatwoot handler
chatwoot = ChatwootHandler(
    api_url=CHATWOOT_API_URL,
    api_key=CHATWOOT_API_KEY,
    account_id=CHATWOOT_ACCOUNT_ID
)

# Test connection
try:
    teams = await chatwoot.get_teams()
    logger.info(f"Successfully connected to Chatwoot. Found {len(teams)} teams.")
except Exception as e:
    logger.error(f"Failed to connect to Chatwoot: {e}")
    raise

In [ ]:
# Define regions list
import json
with open('non_pipe_configurations/regions.json', 'r') as f:
    regions = json.load(f)

logger.info(f"Setting up regions custom attribute with {len(regions)} regions")

In [ ]:
# Create the custom attribute definition
custom_attributes = [
    {
        "display_name": "Region",
        "attribute_key": "region",
        "attribute_values": regions,
        "description": "Geographic region for the conversation",
        "attribute_model": 0,  # 0 for conversation attribute
        "attribute_display_type": 6,  # 6 for list type
    },
    {
        "display_name": "Region Original Input",
        "attribute_key": "region_original_input",
        "description": "Original text input for region",
        "attribute_model": 0,  # 0 for conversation attribute
        "attribute_display_type": 1,  # 1 for text type
    },
]

# Create custom attributes
for attr in custom_attributes:
    try:
        # Check if attribute already exists
        url = f"{chatwoot.account_url}/custom_attribute_definitions"
        async with httpx.AsyncClient() as client:
            response = await client.get(url, headers=chatwoot.headers)
            response.raise_for_status()
            existing_attrs = response.json()

            # Check if attribute with this key exists
            attr_exists = any(a["attribute_key"] == attr["attribute_key"] for a in existing_attrs)

            if not attr_exists:
                # Create attribute if it doesn't exist
                data = {
                    "attribute_display_name": attr["display_name"],
                    "attribute_display_type": attr["attribute_display_type"],
                    "attribute_description": attr["description"],
                    "attribute_key": attr["attribute_key"],
                    "attribute_model": attr["attribute_model"],
                }

                # Add attribute_values only for list type
                if attr["attribute_display_type"] == 6:
                    data["attribute_values"] = attr["attribute_values"]

                response = await client.post(url, json=data, headers=chatwoot.headers)
                response.raise_for_status()
                print(f"Created custom attribute: {attr['display_name']}")
            else:
                print(f"Custom attribute already exists: {attr['display_name']}")

    except Exception as e:
        print(f"Failed to create custom attribute {attr['display_name']}: {e}")

In [ ]:
# Define teams
with open("non_pipe_configurations/teams.json", "r") as f:
    teams = json.load(f)

# Create teams
for team in teams:
    try:
        # First check if team exists
        existing_teams = await chatwoot.get_teams()
        team_exists = any(t["name"] == team["name"] for t in existing_teams)

        if not team_exists:
            # Create team if it doesn't exist
            url = f"{chatwoot.account_url}/teams"
            data = {
                "name": team["name"],
                "description": team["description"],
                "allow_auto_assign": team["allow_auto_assign"],
                "private": team["private"],
            }

            async with httpx.AsyncClient() as client:
                response = await client.post(url, json=data, headers=chatwoot.headers)
                response.raise_for_status()
                print(f"Created team: {team['name']}")
        else:
            print(f"Team already exists: {team['name']}")

    except Exception as e:
        print(f"Failed to create team {team['name']}: {e}")